<a href="https://colab.research.google.com/github/babatundeibukun/DEEP-LEARNING-PROJECT---COMPUTER/blob/main/solving_data_imbalance_with_smote_method_and_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [4]:
df  = pd.read_csv('/content/Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
df.shape

(10000, 14)

In [6]:
df.dtypes

,0
RowNumber,int64
CustomerId,int64
Surname,object
CreditScore,int64
Geography,object
Gender,object
Age,int64
Tenure,int64
Balance,float64
NumOfProducts,int64


In [7]:
#I need to drop the columns that are not needed

df1 = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis = 1 )
df1.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
df1.dtypes

,0
CreditScore,int64
Geography,object
Gender,object
Age,int64
Tenure,int64
Balance,float64
NumOfProducts,int64
HasCrCard,int64
IsActiveMember,int64
EstimatedSalary,float64


In [9]:
df1['Gender'].unique()

array(['Female', 'Male'], dtype=object)

In [10]:
df1['Gender'] = df1['Gender'].replace({'Female' : 0, 'Male' : 1})

<ipython-input-10-5ed6f3f3a3eb>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1['Gender'] = df1['Gender'].replace({'Female' : 0, 'Male' : 1})


In [11]:
df1 = pd.get_dummies(df1, 'Geography', drop_first = True, dtype= int)
df1.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,1,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0,1


#Realized that I need some scaling

In [12]:
from sklearn.preprocessing import MinMaxScaler
sx = MinMaxScaler()


cols_to_scale = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

df1[cols_to_scale] = sx.fit_transform(df[cols_to_scale])

df1[cols_to_scale].head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,EstimatedSalary
0,0.538,0.324324,0.2,0.000000,0.000000,0.506735
1,0.516,0.310811,0.1,0.334031,0.000000,0.562709
2,0.304,0.324324,0.8,0.636357,0.666667,0.569654
3,0.698,0.283784,0.1,0.000000,0.333333,0.469120
4,1.000,0.337838,0.2,0.500246,0.000000,0.395400


In [13]:
df1.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain
0,0.538,0,0.324324,0.2,0.000000,0.000000,1,1,0.506735,1,0,0
1,0.516,0,0.310811,0.1,0.334031,0.000000,0,1,0.562709,0,0,1
2,0.304,0,0.324324,0.8,0.636357,0.666667,1,0,0.569654,1,0,0
3,0.698,0,0.283784,0.1,0.000000,0.333333,0,0,0.469120,0,0,0
4,1.000,0,0.337838,0.2,0.500246,0.000000,1,1,0.395400,0,0,1


#Checking if there is an imbalance in the outcome

In [14]:
df['Exited'].value_counts()

,count
Exited,
0,7963
1,2037


In [15]:
#There is huge imbalance in the dataset, I can solve that with oversampling the minority class, undersampling the majority class, smote or ensemble orr focal loss,
# So i will try all of them one after the other but starting wiht smote since i did undersampling the majority class already .



In [16]:
X = df1.drop('Exited', axis = 1)
X.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain
0,0.538,0,0.324324,0.2,0.000000,0.000000,1,1,0.506735,0,0
1,0.516,0,0.310811,0.1,0.334031,0.000000,0,1,0.562709,0,1
2,0.304,0,0.324324,0.8,0.636357,0.666667,1,0,0.569654,0,0
3,0.698,0,0.283784,0.1,0.000000,0.333333,0,0,0.469120,0,0
4,1.000,0,0.337838,0.2,0.500246,0.000000,1,1,0.395400,0,1


In [17]:
y = df1['Exited']

#SMOTE - METHOD

In [18]:
#first i need to divide the data set into train and test split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( X, y , test_size = 0.2 , random_state = 1234, stratify = y)




In [19]:
X_train.shape

(8000, 11)

In [20]:
X_test.shape

(2000, 11)

In [21]:
y_train.value_counts()

,count
Exited,
0,6370
1,1630


In [22]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy = 'minority')
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)


In [23]:
y_resampled.value_counts()

,count
Exited,
0,6370
1,6370


In [25]:
X_resampled.shape

(12740, 11)

In [30]:
import tensorflow as tf
from tensorflow import keras

model =  keras.Sequential([
    keras.layers.Dense(12740, input_shape = (11,), activation = 'relu'),
    keras.layers.Dense(100,  activation = 'relu'),
    keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile( optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'] )

model.fit(X_resampled, y_resampled, epochs = 15)

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


399/399 ━━━━━━━━━━━━━━━━━━━━ 12s 27ms/step - accuracy: 0.6850 - loss: 0.5872
Epoch 2/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step - accuracy: 0.7583 - loss: 0.4855
Epoch 3/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.7788 - loss: 0.4584
Epoch 4/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - accuracy: 0.7856 - loss: 0.4542
Epoch 5/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.7964 - loss: 0.4278
Epoch 6/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step - accuracy: 0.8036 - loss: 0.4269
Epoch 7/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 11s 26ms/step - accuracy: 0.8096 - loss: 0.4147
Epoch 8/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step - accuracy: 0.8074 - loss: 0.4107
Epoch 9/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 19s 25ms/step - accuracy: 0.8175 - loss: 0.3967
Epoch 10/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step - accuracy: 0.8151 - loss: 0.4020
Epoch 11/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 10s 26ms/step - accuracy: 0.8244 - loss: 0.3858
Epoch 12/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 9s

#Now Lets Evaluate

In [31]:
model.evaluate(X_test, y_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7868 - loss: 0.4944


[0.49502983689308167, 0.7764999866485596]

In [34]:
y_preds = np.round(model.predict(X_test))
y_preds


63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step


array([[1.],
       [1.],
       [1.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [35]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

           0       0.92      0.79      0.85      1593
           1       0.47      0.73      0.57       407

    accuracy                           0.78      2000
   macro avg       0.69      0.76      0.71      2000
weighted avg       0.83      0.78      0.79      2000



The above result still looks terrible to me though

#I want to try to use Ensemble

In [38]:
df2 = df1.copy()
df2.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain
0,0.538,0,0.324324,0.2,0.000000,0.000000,1,1,0.506735,1,0,0
1,0.516,0,0.310811,0.1,0.334031,0.000000,0,1,0.562709,0,0,1
2,0.304,0,0.324324,0.8,0.636357,0.666667,1,0,0.569654,1,0,0
3,0.698,0,0.283784,0.1,0.000000,0.333333,0,0,0.469120,0,0,0
4,1.000,0,0.337838,0.2,0.500246,0.000000,1,1,0.395400,0,0,1


In [39]:
X = df2.drop('Exited', axis = 1)
X.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain
0,0.538,0,0.324324,0.2,0.000000,0.000000,1,1,0.506735,0,0
1,0.516,0,0.310811,0.1,0.334031,0.000000,0,1,0.562709,0,1
2,0.304,0,0.324324,0.8,0.636357,0.666667,1,0,0.569654,0,0
3,0.698,0,0.283784,0.1,0.000000,0.333333,0,0,0.469120,0,0
4,1.000,0,0.337838,0.2,0.500246,0.000000,1,1,0.395400,0,1


In [40]:
y = df2['Exited']
y.head()

,Exited
0,1
1,0
2,1
3,0
4,0


In [43]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train , y_test =  train_test_split(X, y, test_size = 0.2 , random_state = 1234,stratify = y )

In [49]:
y_train.value_counts()

,count
Exited,
0,6370
1,1630


#USING ENSEMBLE

In [61]:
class_0 = df2[df2['Exited'] == 0]
class_1 = df2[df2['Exited'] == 1]

In [62]:
count_0, count_1 = class_0.shape[0], class_1.shape[0]
count_0, count_1

(7963, 2037)

In [63]:
class_0[0:2037].shape

(2037, 12)

In [ ]:
def get_train_batch(start, end, maj_class, min_class):
  df_train = pd.concat([maj_class[start, end], min_class] , axix = 0)
